## Imports and drive

In [1]:
from keras import Model
from keras import backend
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from keras.layers import Dense, Input, Conv2D, Concatenate, Flatten, BatchNormalization, Embedding, Activation, Dropout, LeakyReLU, ReLU, Reshape, Conv2DTranspose, concatenate, MaxPooling2D
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications.inception_v3 import InceptionV3, preprocess_input

import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import os, time, random, zipfile, cv2
import pandas as pd

from google.colab import files
from urllib import request
from google.colab import drive
from scipy.linalg import sqrtm
from PIL import Image, ImageOps

print(tf.__version__)

2.4.1


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## Paths

In [40]:
dataset_folder = os.path.join(os.getcwd(), "Dataset")
dataset_path = os.path.join(dataset_folder, "celeba.zip")
extract_path = os.path.join(dataset_folder, "img_align_celeba")
cropped_path = os.path.join(dataset_folder, "Cropped")
attributes_path = os.path.join(dataset_folder, "list_attr_celeba.csv")
generated_images_path = os.path.join(os.getcwd(), "Generated images")
generator_path = os.path.join(os.getcwd(), "Generators")
discriminator_path = os.path.join(os.getcwd(), "Discriminators")
gan_path = os.path.join(os.getcwd(), "GAN")
g_model_path = os.path.join(generator_path, "generator.h5")
g_model_path_drive = os.path.join("/content/drive/MyDrive/Generators", "generator.h5")
d_model_path = os.path.join(discriminator_path, "discriminator.h5")
d_model_path_drive = os.path.join("/content/drive/MyDrive/Discriminators", "discriminator.h5")
gan_model_path = os.path.join(generator_path, "gan.h5")
gan_model_path_drive = os.path.join("/content/drive/MyDrive/GAN", "gan.h5")

if not os.path.exists(generator_path):
  os.makedirs(generator_path)

if not os.path.exists(discriminator_path):
  os.makedirs(discriminator_path)

if not os.path.exists(gan_path):
  os.makedirs(gan_path)

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

if not os.path.exists(cropped_path):
    os.makedirs(cropped_path)

if not os.path.exists(generated_images_path):
    os.makedirs(generated_images_path)

dataset_url =  "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip"

In [4]:
!cp '/content/drive/MyDrive/list_attr_celeba.csv' '/content/Dataset'

## Dataset download and preprocessing

In [5]:
def download_dataset(download_path, url):
  if not os.path.exists(download_path):
    print("Downloading dataset...", end=" ")
    request.urlretrieve(url, download_path)
    print("Done!")
  else:
    print("Already downloaded")

def extract_dataset(download_path, dataset_folder, extract_path):
  if not os.path.exists(extract_path):
    print("Extracting dataset...", end=" ")
    with zipfile.ZipFile(download_path, "r") as zip_ref:
        zip_ref.extractall(dataset_folder)
    print("Done!")
  else:
    print("Already extracted")

# Download
download_dataset(dataset_path, dataset_url)

# Extraction
extract_dataset(dataset_path, dataset_folder, extract_path)

Extracting dataset... Done!


In [6]:
def crop(source_path, dest_path):
  if not os.path.isdir(dest_path):
    os.mkdir(dest_path)
  if len(os.listdir(dest_path)) == 0:
    images = os.listdir(source_path)
    for img_path in images:
      img = Image.open(source_path + "/" + img_path)
      f, e = os.path.splitext(img_path)
      imCrop = img.crop((25, 45, 128 + 25, 128 + 45)) 
      imCrop.save(dest_path + "/" + f + '.jpg', "JPEG", quality=100)

crop(extract_path, cropped_path)

In [7]:
def load_attributes(path):
  df = pd.read_csv(path)
  ids = df['image_id'] 
  attributes = df.drop(columns=['image_id'])
  attributes.replace(to_replace=-1, value=0, inplace=True)
  attributes = attributes.apply(lambda x: x.to_numpy(), axis=1)
  df = pd.DataFrame(columns = ['file_id', 'attributes'])
  df['file_id'] = ids
  df['attributes'] = attributes
  return df
  
attributes = load_attributes(attributes_path)
attributes_list = pd.read_csv(attributes_path).drop(columns=['image_id']).columns
attributes_dict = dict(enumerate(attributes_list))

In [8]:
def create_custom_label(attr):
  label = []
  for k,v in attributes_dict.items():
    if v in attr:
      label.append(1)
    else:
      label.append(0)
  return np.array(label)

In [9]:
def decode_label(label):
  attr = []
  for i,element in enumerate(label):
    if element == 1:
      attr.append(attributes_dict[i].replace("'", ""))
  return ', '.join(attr)

In [ ]:
def concat_images(images, size, shape=None):
    # Open images and resize them
    width, height = size
  
    # Create canvas for the final image with total size
    shape = shape if shape else (1, len(images))
    image_size = (width * shape[1], height * shape[0])
    image = Image.new('RGB', image_size)
    
    # Paste images into final image
    for row in range(shape[0]):
        for col in range(shape[1]):
            offset = width * col, height * row
            idx = row * shape[1] + col
            image.paste(images[idx], offset)
    
    return image

# Get list of image paths
image_paths = [os.path.join(cropped_path, f) 
               for f in os.listdir(cropped_path) if f.endswith('.jpg')]

# Random selection of images
image_array = random.choices(image_paths, k=84)

images = map(Image.open, image_array)
images = [ImageOps.fit(image, (64, 64), Image.ANTIALIAS) 
              for image in images]

# Create and save image grid
image = concat_images(images, (64, 64), (8, 8))
plt.figure(figsize = (16,16))
plt.imshow(image)

In [11]:
def load_dataset(cropped_path, attributes, batch_size, image_shape, subset=False):
    dataset_generator = ImageDataGenerator()
    if subset:
      dataset_generator = dataset_generator.flow_from_dataframe(
        dataframe=attributes[0:20000], directory=cropped_path, target_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size, x_col='file_id', y_col='attributes', class_mode='raw')
    else:
      dataset_generator = dataset_generator.flow_from_dataframe(
        dataframe=attributes, directory=cropped_path, target_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size, x_col='file_id', y_col='attributes', class_mode='raw')

    return dataset_generator

## Gan section

### Function to save generated images

In [12]:
def save_generated_images(generated_images, epoch, batch_number, batch_size):

  plt.figure(figsize=(8, 8), num=2)
  gs1 = gridspec.GridSpec(8, 8)
  gs1.update(wspace=0, hspace=0)

  for i in range(batch_size):
      ax1 = plt.subplot(gs1[i])
      ax1.set_aspect('equal')
      image = generated_images[i, :, :, :]
      image += 1
      image *= 127.5
      fig = plt.imshow(image.astype(np.uint8))
      plt.axis('off')
      fig.axes.get_xaxis().set_visible(False)
      fig.axes.get_yaxis().set_visible(False)
      
  plt.tight_layout()
  save_name = 'Generated images/epoch_' + str(
      epoch + 1) + '_batch_' + str(batch_number + 1) + '.png'

  plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
  plt.pause(0.0000000001)
  plt.show()

### Wasserstein loss

In [13]:
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

### Focal loss

In [14]:
def binary_focal_loss(y_true, y_pred):
  """
  :param y_true: A tensor of the same shape as `y_pred`
  :param y_pred:  A tensor resulting from a sigmoid
  :return: Output tensor.
  """
  gamma=2.
  alpha=.25
  
  y_true = tf.cast(y_true, tf.float32)
  # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
  epsilon = backend.epsilon()

  # Clip the prediciton value
  y_pred = backend.clip(y_pred, epsilon, 1.0 - epsilon)
  
  # Calculate p_t
  p_t = tf.where(backend.equal(y_true, 1), y_pred, 1 - y_pred)
  
  # Calculate alpha_t
  alpha_factor = backend.ones_like(y_true) * alpha
  alpha_t = tf.where(backend.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
  
  # Calculate cross entropy
  cross_entropy = -backend.log(p_t)
  weight = alpha_t * backend.pow((1 - p_t), gamma)
  
  # Calculate focal loss
  loss = weight * cross_entropy
  
  # Sum the losses in mini_batch
  loss = backend.mean(backend.sum(loss, axis=1))
  
  return loss

### FID function

In [15]:
def scale_images(images, new_shape):
  images_list = list()

  for image in images:
    # resize with bilinear interpolation
	  new_image = cv2.resize(image, (new_shape, new_shape))
    # store
	  images_list.append(new_image)
  return np.asarray(images_list)

def calculate_fid(model, images1, images2):
  # scale images
  images1 = scale_images(images1, 299)
  images2 = scale_images(images2, 299)

  # calculate activations
  act1 = model.predict(images1)
  act2 = model.predict(images2)

  # calculate mean and covariance statistics
  mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

  # calculate sum squared difference between means
  ssdiff = np.sum((mu1 - mu2)**2.0)

  # calculate sqrt of product between cov
  covmean = sqrtm(sigma1.dot(sigma2))

  # check and correct imaginary numbers from sqrt
  if np.iscomplexobj(covmean):
    covmean = covmean.real

  # calculate score
  fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

### Real and Fake data generation

In [16]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
  # generate points in the latent space
  x_input = np.random.randn(latent_dim * n_samples)
  
  # reshape into a batch of inputs for the network
  x_input = x_input.reshape(n_samples, latent_dim)
  
  # generate labels
  labels = np.array(random.sample(list(attributes['attributes']), n_samples))
  
  return [x_input, labels]

def generate_real_samples(dataset):
  X, label = dataset.next()
  X /= 127.5
  X -= 1
  y = -np.ones(X.shape[0])
  return [X, label.tolist()], y

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples, custom_label=None):
  
  # generate points in latent space
  x_input, labels = generate_latent_points(latent_dim, n_samples)
  
  if custom_label is not None:
    labels = np.tile(custom_label, (n_samples,1))
  
  # predict outputs
  X = generator.predict([x_input, labels])
  
  # create class labels
  y = np.ones((n_samples, 1))
  
  return [X, labels], y

### Spectral Normalization and Clipping Constraint

In [ ]:
class SpectralNormalization(tf.keras.layers.Wrapper):
    """Performs spectral normalization on weights.
    This wrapper controls the Lipschitz constant of the layer by
    constraining its spectral norm, which can stabilize the training of GANs.
    See [Spectral Normalization for Generative Adversarial Networks](https://arxiv.org/abs/1802.05957).
    """

    def __init__(self, layer: tf.keras.layers, power_iterations: int = 1, **kwargs):
        super().__init__(layer, **kwargs)
        if power_iterations <= 0:
            raise ValueError(
                "`power_iterations` should be greater than zero, got "
                "`power_iterations={}`".format(power_iterations)
            )
        self.power_iterations = power_iterations
        self._initialized = False

    def build(self, input_shape):
        """Build `Layer`"""
        super().build(input_shape)
        input_shape = tf.TensorShape(input_shape)
        self.input_spec = tf.keras.layers.InputSpec(shape=[None] + input_shape[1:])

        if hasattr(self.layer, "kernel"):
            self.w = self.layer.kernel
        elif hasattr(self.layer, "embeddings"):
            self.w = self.layer.embeddings
        else:
            raise AttributeError(
                "{} object has no attribute 'kernel' nor "
                "'embeddings'".format(type(self.layer).__name__)
            )

        self.w_shape = self.w.shape.as_list()

        self.u = self.add_weight(
            shape=(1, self.w_shape[-1]),
            initializer=tf.initializers.TruncatedNormal(stddev=0.02),
            trainable=False,
            name="sn_u",
            dtype=self.w.dtype,
        )

    def call(self, inputs, training=None):
        """Call `Layer`"""
        if training is None:
            training = tf.keras.backend.learning_phase()

        if training:
            self.normalize_weights()

        output = self.layer(inputs)
        return output

    def compute_output_shape(self, input_shape):
        return tf.TensorShape(self.layer.compute_output_shape(input_shape).as_list())

    @tf.function
    def normalize_weights(self):
        """Generate spectral normalized weights.
        This method will update the value of `self.w` with the
        spectral normalized value, so that the layer is ready for `call()`.
        """

        w = tf.reshape(self.w, [-1, self.w_shape[-1]])
        u = self.u

        with tf.name_scope("spectral_normalize"):
            for _ in range(self.power_iterations):
                v = tf.math.l2_normalize(tf.matmul(u, w, transpose_b=True))
                u = tf.math.l2_normalize(tf.matmul(v, w))

            sigma = tf.matmul(tf.matmul(v, w), u, transpose_b=True)

            self.w.assign(self.w / sigma)
            self.u.assign(u)

    def get_config(self):
        config = {"power_iterations": self.power_iterations}
        base_config = super().get_config()
        return {**base_config, **config}

In [ ]:
class ClipConstraint(Constraint):
  # set clip value when initialized
  def __init__(self, clip_value):
    self.clip_value = clip_value

  # clip model weights to hypercube
  def __call__(self, weights):
    return backend.clip(weights, -self.clip_value, self.clip_value)

  # get the config
  def get_config(self):
    return {'clip_value': self.clip_value}

const = ClipConstraint(0.01)

### Discriminator

Tried to remove kernel clip constraint and add Spectral Normalization

In [35]:
def define_discriminator(in_shape=(64,64,3), label_shape=(40,), n_attributes=40):

  init = RandomNormal(stddev=0.02)

  input_image = Input(shape=in_shape)

  # define the constraint
  const = ClipConstraint(0.01)

	# downsample
  d = SpectralNormalization(Conv2D(32, (4,4), strides=(2,2), padding='same', input_shape=in_shape, kernel_initializer=init))(input_image)
  d = LeakyReLU(alpha=0.2)(d)
  d = Dropout(0.2)(d)
	
  # downsample
  d = SpectralNormalization(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Dropout(0.2)(d)
  
  # downsample
  d = SpectralNormalization(Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Dropout(0.2)(d)
  
  # downsample
  d = SpectralNormalization(Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))(d)
  d = LeakyReLU(alpha=0.2)(d)
  d = Dropout(0.2)(d)
  
  # classifier
  d = Flatten()(d)
  d = Dropout(0.4)(d)
  
  out1 = Dense(1, activation='linear')(d)
  out2 = Dense(n_attributes, activation='sigmoid')(d)
	
  # compile model
  opt = RMSprop(lr=0.00005)

  model = Model(input_image,  [out1, out2])
  
  model.compile(loss=[wasserstein_loss, binary_focal_loss], optimizer=opt, metrics=['accuracy'])
  return model

### Generator

In [ ]:
def define_generator(latent_dim, n_attributes=40):
  input_label = Input(shape=(40,))

  init = RandomNormal(stddev=0.02)
  
  image_input = Input(shape=(latent_dim,))
  
  merged_input = Concatenate()([image_input, input_label])  
  
  hid = Dense(140 * 8 * 8, activation='relu')(merged_input)    
  hid = BatchNormalization(momentum=0.8)(hid)
  hid = LeakyReLU(alpha=0.2)(hid)
  hid = Reshape((8, 8, 140))(hid)
  
  # upsample to 16x16
  conv = Conv2DTranspose(256, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(hid)
  conv = BatchNormalization(momentum=0.8)(conv)
  conv = LeakyReLU(alpha=0.2)(conv)
  
  # upsample to 32x32
  conv = Conv2DTranspose(128, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(conv)
  conv = BatchNormalization(momentum=0.8)(conv)
  conv = LeakyReLU(alpha=0.2)(conv)
  
  # upsample to 64x64
  conv = Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(conv)
  conv = BatchNormalization(momentum=0.8)(conv)
  conv = LeakyReLU(alpha=0.2)(conv)
  
  # generate
  act = Conv2D(3, (16,16), activation='tanh', padding='same', kernel_initializer=init)(conv)
  
  model = Model([image_input, input_label], act)
  
  return model

### GAN

In [43]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	
  # make weights in the discriminator not trainable
  for layer in discriminator.layers:
    if not isinstance(layer, BatchNormalization):
      layer.trainable = False

  gen_image, gen_label = generator.input

  gen_output = generator.output
  gan_output = discriminator(gen_output)

  model = Model([gen_image, gen_label], gan_output)
  
	# compile model
  opt = RMSprop(lr=0.00005)
  model.compile(loss=[wasserstein_loss, binary_focal_loss], optimizer=opt, metrics=['accuracy'])
  return model

## Train function

In [44]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, inception_model, dataset, latent_dim, n_epochs=20, n_batch=64):
    bat_per_epo = int(202599 / n_batch)
    # manually enumerate epochs
    for i in range(n_epochs):
      start = time.time()

      # enumerate batches over the training set
      for j in range(bat_per_epo):

        # train discriminator 3 times as the generator
        for k in range(3):
          # get randomly selected 'real' samples
          [X_real, labels_real], y_real = generate_real_samples(dataset)
          current_batch_size = X_real.shape[0]
          
          labels_real = np.array(labels_real).reshape(current_batch_size, 40)
          # update discriminator model weights
          d1_history = d_model.train_on_batch(X_real, [y_real, labels_real])
                    
          # generate 'fake' examples
          [X_fake, labels_fake], y_fake = generate_fake_samples(
              g_model, latent_dim, current_batch_size)
          
          # update discriminator model weights
          d2_history = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
          
        # prepare points in latent space as input for the generator
        X_gan, label_gan = generate_latent_points(latent_dim, current_batch_size)
        # create inverted labels for the fake samples
        y_gan = -np.ones((current_batch_size, 1))
        
        # update the generator via the discriminator's error
        g_history = gan_model.train_on_batch([X_gan, label_gan], [y_gan, label_gan])
        # summarize loss on this batch
        if ((j + 1) % 200 == 0):
          print('Epoch %d\tbatch %d/%d\td1=%.3f\td2=%.3f\tg=%.3f' %
              (i+1, j+1, bat_per_epo, d1_history[0], d2_history[0], g_history[0]))

        if ((j + 1) % 1000 == 0):
            
            random_idx = np.random.randint(0, 202599)
            custom_label = attributes['attributes'][random_idx]
            
            # choose a label with at least 8 attributes
            while(len(custom_label[custom_label == 1]) < 8):
              random_idx = np.random.randint(0, 202599)
              custom_label = attributes['attributes'][random_idx]
            
            [fake, label_fake], _ = generate_fake_samples(generator, latent_dim, 64, custom_label=custom_label)
            
            print('\n', decode_label(list(custom_label)))
            save_generated_images(fake, i, j, n_batch)
      
      end = time.time()
      print("Elapsed time: {} minutes and {:.2f} seconds".format(int((end-start)//60), (end-start)%60))
    
      # save the models
      g_model.save_weights(g_model_path_drive)
      d_model.save_weights(d_model_path_drive)
 
# size of the latent space
latent_dim = 100

discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)

load_weights = False

if load_weights:

  print("Resuming training...", end=" ")
  discriminator.load_weights(d_model_path_drive)
  generator.load_weights(g_model_path_drive)
  print("Weights loaded")

# load image data
dataset = load_dataset(cropped_path, attributes, 64, (64,64,3))

# prepare the inception model
inception = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

Found 202599 validated image filenames.


## Train

In [ ]:
# train model
train(generator, discriminator, gan_model, inception, dataset, latent_dim, n_epochs=15, n_batch=64)

Epoch 1	batch 200/3165	d1=-151.265	d2=-137.506	g=143.882
Epoch 1	batch 400/3165	d1=-276.327	d2=-262.765	g=269.316


# Test

## FID value

In [29]:
real_for_fid = []
fake_for_fid = []

IMAGES_FOR_FID = 10000

while len(real_for_fid) < IMAGES_FOR_FID:
  [real, label_real], _ = generate_real_samples(dataset)
  [fake, label_fake], _ = generate_fake_samples(generator, latent_dim, 64)
  
  for image in real:
    real_for_fid.append(image)
  for image in fake:
    fake_for_fid.append(image)
  
  if len(real_for_fid) > IMAGES_FOR_FID:
    real_for_fid = real_for_fid[:IMAGES_FOR_FID]
    fake_for_fid = fake_for_fid[:IMAGES_FOR_FID]

  print("\r{}/{}".format(len(real_for_fid), 10000), end="")

real_images = np.array(real_for_fid)
fake_images = np.array(fake_for_fid)
fid = calculate_fid(inception, real, fake)

print("\n\nFid score calculated on {} real images and {} fake images: {:.2f}".format(real_images.shape[0], fake_images.shape[0], fid))

10000/10000

Fid score calculated on 10000 real images and 10000 fake images: 173.73


## Custom label generation

.         |           |    |           |    |           |    |         .
----------|-----------|----|-----------|----|-----------|----|----------
5_o_Clock_Shadow | Blurry | Male | Sideburns
Arched_Eyebrows | Brown_Hair | Mouth_Slightly_Open | Smiling
Attractive | Bushy_Eyebrows | Mustache | Straight_Hair
Bags_Under_Eyes | Chubby | Narrow_Eyes | Wavy_Hair
Bald | Double_Chin | No_Beard | Wearing_Earrings
Bangs | Eyeglasses | Oval_Face | Wearing_Hat
Big_Lips | Goatee | Pale_Skin | Wearing_Lipstick
Big_Nose | Gray_Hair | Pointy_Nose | Wearing_Necklace
Black_Hair | Heavy_Makeup | Receding_Hairline | Wearing_Necktie
Blond_Hair | High_Cheekbones | Rosy_Cheeks | Young

In [ ]:
[fake, label_fake], _ = generate_fake_samples(generator, latent_dim, 64)
print("Different attributes per face")
save_generated_images(fake, 0, 0, 64)

In [ ]:
print("Custom attributes")
for i in range(10):
  random_idx = np.random.randint(0, 202599)
  custom_label = attributes['attributes'][random_idx]
            
  # choose a label with at least 8 attributes
  while(len(custom_label[custom_label == 1]) < 8):
    random_idx = np.random.randint(0, 202599)
    custom_label = attributes['attributes'][random_idx]  print(decode_label(custom_label))
  
  [real, label_real], _ = generate_real_samples(dataset)
  [fake, label_fake], _ = generate_fake_samples(generator, latent_dim, 64, custom_label=custom_label)
  save_generated_images(fake, 0, 0, 64)
  print("\n\n")